| Version | Datum       | Bearbeiter | Anmerkung          |
|---------|-------------|------------|--------------------|
| 1.0     | 27.02.2025  | Erhard Rainer  | Erste Version      |
| 1.1     | 28.08.2025  | Erhard Rainer | Übertragung in Jupiter Notebook |


# SharePoint 2 SQL (klassische Benutzername und Passwort Anmeldung)

## Was macht das Skript?

Es liest eine Konfigurationstabelle aus SQL, holt auf Basis dieser Konfiguration Einträge aus **SharePoint-Listen**, transformiert die Spalten gemäß Mapping und schreibt die Daten in **SQL-Tabellen** (optional mit vorherigem `TRUNCATE` und optionalem SP-Nachlauf). Für jeden Konfigurationseintrag läuft derselbe ETL-Zyklus.

## Ablauf (High Level)

1. **Config laden** (`config.json`): Nutzer/Passwort für SharePoint sowie SQL (Server, User, Passwort).
    
2. **Konfiguration aus SQL lesen**: `SELECT * FROM [config].[Sharepoint] WHERE Disabled = 0` (DB standardmäßig `BI_RAW`).
    
3. **Pro Konfigurationseintrag:**
    
    - Parameter holen: `SITE_URL`, `LIST_TITLE`, Ziel-DB/Schema/Tabelle, Spalten-Mapping (`Source_columns` → `Translated_columns`), optional `SP`.
        
    - **SharePoint lesen**:
        
        - Login mit Benutzername/Passwort.
            
        - Liste per Paging in **Batches à 500** Items abrufen.
            
        - Spalten laut Mapping umbenennen; fehlende Spalten werden einmalig pro Name gewarnt.
            
        - `Anzeigeseite` (DispForm-URL) aus `site_url`, `list_title`, `ID` erzeugen.
            
    - **SQL-Ziel initialisieren** (SQLAlchemy/ODBC):
        
        - Tabelle bei Bedarf **anlegen** (Datentypen aus `pandas`\-DTypes abgeleitet).
            
        - Optional **leeren** (`TRUNCATE`) und dann per `to_sql` **anhängen**.
            
        - Optional **Stored Procedure** ausführen (`EXEC <SP>`).
            
4. **Logging auf Konsole**, dann nächster Eintrag / Ende.
    

## Welche Authentifizierung verwendet es?

- **SharePoint Online**:
    
    - `office365-sharepoint` mit `UserCredential(username, password)` → **klassische Benutzername/Passwort-Anmeldung** (delegierte Benutzeridentität; kein App-Registrat, kein MSAL/OAuth-Client-Credentials).
        
    - Hinweis: Das ist praktisch, aber sicherheitlich/operativ schwächer; Microsoft empfiehlt heute App-Only (Zertifikat/Secret) über **Microsoft Graph**.
        
- **SQL Server**:
    
    - **SQL-Authentifizierung** (User/Pass) in der ODBC-Connection (`mssql+pyodbc`, **ODBC Driver 17**).
        

## Erwartete Spalten in `[config].[Sharepoint]`

Mindestens:  
`SITE_URL`, `LIST_TITLE`, `DB_NAME`, `DB_SCHEMA`, `DB_TABLE`, `Source_columns`, `Translated_columns`, `Disabled` (0/1), optional `SP`.

## Besonderheiten / Edge Cases

- Falls die Liste **leer** ist → leeres DataFrame, kein Insert.
    
- Wenn `Source_columns` ≠ `Translated_columns` in der Länge → **Fehler**.
    
- Wenn keine `ID` in den Daten → Abbruch (für die DispForm-URL nötig).
    
- Datentypen werden heuristisch aus `pandas`\-DTypes abgeleitet (z. B. `int64` → `INTEGER`, `datetime` → `DateTime`, sonst `String`).
    

## Kurzer Sicherheitshinweis (empfohlen)

- Secrets nicht im Klartext: z. B. **Azure Key Vault**, Umgebungsvariablen, `.env`.
    
- Für SharePoint: möglichst auf **App-Only über MSAL/Graph** (Client-ID/Secret oder Zertifikat) umstellen; Least Privilege.
    
- ODBC **Driver 18** mit `Encrypt=yes;TrustServerCertificate=no` erwägen.

In [ ]:
%pip install --upgrade pandas sqlalchemy pyodbc Office365-REST-Python-Client
%pip install --upgrade python-dotenv tqdm pyarrow

## Test der Module

In [3]:
import pandas, sqlalchemy, pyodbc
import office365
print("pandas:", pandas.__version__)
print("sqlalchemy:", sqlalchemy.__version__)
print("pyodbc:", pyodbc.version)
print("ODBC-Treiber:", pyodbc.drivers())


pandas: 2.0.3
sqlalchemy: 2.0.43
pyodbc: 5.2.0
ODBC-Treiber: ['SQL Server', 'ODBC Driver 17 for SQL Server', 'ODBC Driver 18 for SQL Server', 'Microsoft Access Driver (*.mdb, *.accdb)', 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)', 'Microsoft Access Text Driver (*.txt, *.csv)', 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)']


## Systemvoraussetzung (nicht per pip)

- **Microsoft ODBC Driver 17/18 for SQL Server** muss am System installiert sein.  
    Falls du **18** nutzt, passe im Connection-String den Treiber an:
    
    - `...driver=ODBC+Driver+18+for+SQL+Server bzw.` `ODBC+Driver+17+for+SQL+Server`.

## Aufbau der Config-Datei

Es gibt auch Prüfskripte, um den Aufbau der Config-Datei zur prüfen siehe [hier](config_json.ipynb).

In [ ]:
{
  "sql": {
    "username": "DEIN_SQL_USER",
    "password": "DEIN_SQL_PASS",
    "server": "Server",
    "DB": "DB",
  },
  "sharepoint": {
    "username": "user@tenant.onmicrosoft.com",
    "password": "DEIN_SP_PASS"
  }
}

# das Skript

In [ ]:
import json
import os
import pandas as pd
import urllib.parse
from sqlalchemy import create_engine, MetaData, Table, Column, text
from sqlalchemy.types import String, Integer, Float, DateTime, Boolean
from office365.sharepoint.client_context import ClientContext
from office365.runtime.auth.user_credential import UserCredential

# --------------------------------------------------------------------------------
# Konfiguration aus config.json laden (mit UTF-8-Encoding) über einen absoluten Pfad
# --------------------------------------------------------------------------------
# Absoluter Pfad zur config.json (anpassen, falls nötig)
config_path = r"C:\python\Scripts\config.json"  
with open(config_path, "r", encoding="utf-8") as f:
    config = json.load(f)

SHAREPOINT_USERNAME = config["sharepoint"]["username"]
SHAREPOINT_PASSWORD = config["sharepoint"]["password"]

SQL_USERNAME = config["sql"]["username"]
SQL_PASSWORD = config["sql"]["password"]
SQL_SERVER = config["sql"]["server"]

# Funktion zur URL-Kodierung der SQL-Zugangsdaten
def encode_sql_credentials(username, password):
    username_enc = urllib.parse.quote_plus(username)
    password_enc = urllib.parse.quote_plus(password)
    return username_enc, password_enc

# --------------------------------------------------------------------------------
# Hilfsfunktionen
# --------------------------------------------------------------------------------

def read_sharepoint_config(db_name="BI_RAW"):
    """
    Liest die Konfiguration aus der Tabelle [config].[Sharepoint] in der angegebenen
    Datenbank (Standard: BI_RAW) und liefert nur die Einträge zurück, bei denen Disabled=0.
    """
    username_enc, password_enc = encode_sql_credentials(SQL_USERNAME, SQL_PASSWORD)
    engine = create_engine(
        f"mssql+pyodbc://{username_enc}:{password_enc}@{SQL_SERVER}/{db_name}?driver=ODBC+Driver+17+for+SQL+Server"
    )
    query = "SELECT * FROM [config].[Sharepoint] WHERE Disabled = 0"
    df_config = pd.read_sql(query, engine)
    return df_config

def get_sharepoint_list_items(site_url, list_title, username, password, source_columns, translated_columns):
    """
    Authentifiziert sich bei SharePoint Online und ruft Elemente aus der angegebenen Liste ab.
    Dabei werden die Einträge in Batches von 500 Elementen abgerufen.
    Die definierten Spalten (source_columns) werden in die übersetzten Spalten (translated_columns)
    überführt und es wird ein DispForm-Link hinzugefügt.
    
    Falls eine erwartete Spalte nicht gefunden wird, wird eine Warnung ausgegeben, die
    alle in dem jeweiligen Listeneintrag verfügbaren Spalten anzeigt.
    """
    try:
        # Authentifizierung und Zugriff auf die Liste
        ctx = ClientContext(site_url).with_credentials(UserCredential(username, password))
        s_list = ctx.web.lists.get_by_title(list_title)
        
        # Alle Listeneinträge in Batches von 500 abrufen
        batch_size = 500
        paged_items = s_list.items.paged(batch_size)
        ctx.load(paged_items)
        ctx.execute_query()
        
        all_items = list(paged_items)
        while paged_items.has_next:
            paged_items = paged_items.get_next()
            ctx.load(paged_items)
            ctx.execute_query()
            all_items.extend(list(paged_items))
        
        print(f"Gesamtanzahl der Listeneinträge: {len(all_items)}")
        
        # Falls die Liste leer ist, direkt ein leeres DataFrame zurückgeben
        if len(all_items) == 0:
            print("Die SharePoint-Liste ist leer. Es werden keine Daten verarbeitet.")
            return pd.DataFrame()
        
        # Überprüfen, ob beide Spaltenlisten gleich lang sind
        if len(source_columns) != len(translated_columns):
            raise ValueError("Die Listen 'source_columns' und 'translated_columns' müssen die gleiche Länge haben.")
        
        # Mapping-Dictionary für die Spaltenübersetzung erstellen
        column_mapping = dict(zip(source_columns, translated_columns))
        
        # Vermerke, welche fehlende Spalte bereits gemeldet wurde (um Dopplungen zu vermeiden)
        missing_reported = {}
        
        # Übersetzen der Listenelemente
        translated_data = []
        for item in all_items:
            translated_item = {}
            for src_col, trans_col in column_mapping.items():
                if src_col in item.properties:
                    translated_item[trans_col] = item.properties.get(src_col, None)
                else:
                    if src_col not in missing_reported:
                        available = list(item.properties.keys())
                        print(f"Warnung: Spalte '{src_col}' nicht gefunden. Verfügbare Spalten: {available}")
                        missing_reported[src_col] = True
                    translated_item[trans_col] = None
            translated_data.append(translated_item)
        
        # DataFrame erstellen
        df = pd.DataFrame(translated_data)
        
        if 'ID' not in df.columns:
            print("Keine 'ID'-Spalte im DataFrame gefunden. Abbruch.")
            return None
        
        # DispForm-Links generieren und als neue Spalte hinzufügen
        df['Anzeigeseite'] = df['ID'].apply(lambda id: generate_dispform_url(site_url, list_title, id))
        
        return df
    except Exception as e:
        print(f"Ein Fehler ist aufgetreten (Liste '{list_title}'): {e}")
        return None

def generate_dispform_url(site_url, list_title, item_id):
    """
    Generiert die URL zur DispForm-Seite eines SharePoint-Listeneintrags.
    """
    encoded_list_title = urllib.parse.quote(list_title)
    dispform_url = f"{site_url}/Lists/{encoded_list_title}/DispForm.aspx?ID={item_id}"
    return dispform_url

def initialize_engine(db_username, db_password, db_server, db_name, db_schema):
    """
    Initialisiert die SQLAlchemy-Engine für die Verbindung zur Datenbank.
    """
    username_enc, password_enc = encode_sql_credentials(db_username, db_password)
    engine = create_engine(
        f'mssql+pyodbc://{username_enc}:{password_enc}@{db_server}/{db_name}?driver=ODBC+Driver+17+for+SQL+Server'
    )
    return engine, db_schema

def create_table_if_not_exists(engine, table_name, df, schema):
    """
    Erstellt die Tabelle, wenn sie nicht existiert, basierend auf dem DataFrame.
    """
    metadata = MetaData(schema=schema)
    columns = []
    for column_name, dtype in zip(df.columns, df.dtypes):
        if dtype == "int64":
            columns.append(Column(column_name, Integer))
        elif dtype == "float64":
            columns.append(Column(column_name, Float))
        elif dtype == "bool":
            columns.append(Column(column_name, Boolean))
        elif str(dtype).startswith("datetime"):
            columns.append(Column(column_name, DateTime))
        else:
            columns.append(Column(column_name, String))
    table = Table(table_name, metadata, *columns, extend_existing=True)
    metadata.create_all(engine)

def insertSQL(engine, table_name, truncate, df, schema, sp_name=None):
    """
    Fügt Daten aus einem DataFrame in eine SQL-Tabelle ein. Optional wird die Tabelle vorher geleert.
    Falls die Tabelle nicht existiert, wird sie automatisch erstellt.
    Nach dem Einfügen kann optional eine Stored Procedure ausgeführt werden.
    """
    try:
        create_table_if_not_exists(engine, table_name, df, schema)
        
        if truncate == 1:
            with engine.begin() as conn:
                conn.execute(text(f"TRUNCATE TABLE {schema}.{table_name}"))
                print(f"Tabelle {schema}.{table_name} wurde geleert.")
        
        if df is None or df.empty:
            print("Keine gültigen Daten zum Einfügen vorhanden.")
            return
        
        df.to_sql(table_name, engine, schema=schema, if_exists='append', index=False)
        print(f"Daten erfolgreich in die Tabelle {schema}.{table_name} eingefügt.")
        
        if sp_name:
            with engine.connect() as conn:
                print(f"Führe Stored Procedure '{sp_name}' aus...")
                conn.execute(text(f"EXEC {sp_name}"))
                print(f"Stored Procedure '{sp_name}' erfolgreich ausgeführt.")
    
    except Exception as e:
        print(f"Ein Fehler ist beim Einfügen in die Tabelle {schema}.{table_name} aufgetreten: {e}")

# --------------------------------------------------------------------------------
# Main-Prozess
# --------------------------------------------------------------------------------

if __name__ == "__main__":
    # 1) Config (in BI_RAW / [config].[Sharepoint]) auslesen
    df_config = read_sharepoint_config(db_name="BI_RAW")
    
    if df_config.empty:
        print("Keine aktiven (Disabled=0) Einträge in [config].[Sharepoint] gefunden.")
        exit()

    # 2) Über jeden aktiven Eintrag iterieren
    for idx, row in df_config.iterrows():
        site_url = row["SITE_URL"]
        list_title = row["LIST_TITLE"]
        
        db_name = row["DB_NAME"]
        db_schema = row["DB_SCHEMA"]
        db_table = row["DB_TABLE"]
        
        source_columns = [col.strip() for col in row["Source_columns"].split(",")]
        translated_columns = [col.strip() for col in row["Translated_columns"].split(",")]
        
        sp_name = row["SP"] if "SP" in row and pd.notnull(row["SP"]) else None

        print(f"\nVerarbeite Eintrag aus [config].[Sharepoint]:")
        print(f" - site_url: {site_url}")
        print(f" - list_title: {list_title}")
        print(f" - db_table: {db_schema}.{db_table}")
        print(f" - Stored Procedure: {sp_name if sp_name else 'Keine'}")
        print(f" - source_columns: {source_columns}")
        print(f" - translated_columns: {translated_columns}")
        
        # 3) Daten aus SharePoint holen
        df_sharepoint = get_sharepoint_list_items(
            site_url=site_url,
            list_title=list_title,
            username=SHAREPOINT_USERNAME,
            password=SHAREPOINT_PASSWORD,
            source_columns=source_columns,
            translated_columns=translated_columns
        )

        if df_sharepoint is None or df_sharepoint.empty:
            print("Die SharePoint-Liste ist leer oder es wurden keine Daten abgerufen.")
            continue
        
        print(f"Anzahl der abgerufenen Elemente: {len(df_sharepoint)}")
        
        # 4) Engine initialisieren für das Ziel-DB (db_name aus Config)
        engine, schema = initialize_engine(
            db_username=SQL_USERNAME,
            db_password=SQL_PASSWORD,
            db_server=SQL_SERVER,
            db_name=db_name,
            db_schema=db_schema
        )
        
        # 5) Daten einfügen (truncate=1 => Tabelle vorher leeren, falls gewünscht)
        insertSQL(engine, db_table, 1, df_sharepoint, schema, sp_name=sp_name)
    
    print("\nVerarbeitung beendet.")

# Fehlerhandling

Die Fehlermeldung „Ein Fehler ist aufgetreten (Liste ‘XY’): An error occurred while retrieving auth cookies from …/\_forms/default.aspx?wa=wsignin1.0“ deutet darauf hin, dass dein Code (über `UserCredential`/CSOM bzw. Office365-REST-Python-Client) versucht, alte **WS-Federation/Forms-Auth**\-Cookies (`FedAuth`/`rtFa`) per **wsignin1.0** zu beziehen – also eine **Legacy-Anmeldung mit Benutzername/Passwort** gegen SharePoint Online. Genau diese Methode gilt heute **nicht mehr als State of the Art** und ist in vielen Tenants durch **Conditional-Access-Policies**, **MFA-Pflicht**, „Block Legacy Authentication“ oder generell deaktivierte WS-Trust/WS-Fed-Flows unterbunden. Ergebnis: Der Cookie-Abruf wird abgewiesen oder in einen interaktiven Flow umgeleitet, den ein Headless-Script nicht durchlaufen kann. Kurz: Solange du `UserCredential(username, password)` nutzt, kollidierst du mit modernen Sicherheitsvorgaben. Der robuste Weg ist **moderne OAuth2/AAD-Authentifizierung**: registriere eine **App** in Entra ID (Azure AD), nutze **App-Only (Client-Credentials)** idealerweise mit **Zertifikat** (statt Secret), erteile **least-privilege**\-Rechte (für Microsoft Graph z. B. **`Sites.Selected`** und die Site explizit freischalten; alternativ SharePoint App-Permissions), hole dir das Token mit **MSAL** und rufe die Daten anschließend über **Graph** (`/sites/{id}/lists/{title}/items?expand=fields`) oder über die **SharePoint REST/CSOM** mit **Bearer-Token** ab (auch der Office365-Client unterstützt `ClientCredential`). Achte darauf, dass keine interaktiven Faktoren (MFA) im Server-Kontext verlangt werden, Secrets/Zertifikate sicher gelagert sind (Key Vault/Secrets Manager), und dass deine Policies (CA, Legacy-Auth-Block) zum App-Only-Szenario passen. In Summe erklärt der Fehler also nicht „falsche Credentials“, sondern dass **passwortbasierte Cookie-Flows** im Tenant **blockiert** sind – eine gewollte Sicherheitsmaßnahme –, und die Lösung ist die **Umstellung auf moderne, nicht-interaktive App-Only-Authentifizierung** mit Token-basiertem Zugriff.